# Import libraries

In [81]:
import numpy as np
import pandas as pd
import urllib
from urllib.parse import urlparse, parse_qsl
import html

import plotly.express as px
import scattertext as st

import re

In [82]:
file_path = "C:\\Users\\tmros\\OneDrive\\Desktop\\Modifying Datasets\\XSS data\\Payloads.csv"
data = pd.read_csv(file_path, encoding='latin1')

## Finding out the basic structure of the data

In [83]:
pd.set_option('display.max_colwidth', None)

print("Shape of the data:", data.shape)
print("Columns in the data:", data.columns.tolist())
print("Data types of the columns:\n", data.dtypes)
print("Number of missing values in each column:\n", data.isnull().sum())

Shape of the data: (43217, 2)
Columns in the data: ['Payloads', 'Class']
Data types of the columns:
 Payloads    object
Class       object
dtype: object
Number of missing values in each column:
 Payloads    0
Class       0
dtype: int64


In [84]:
# Histogram of Class imbalance on original dataset
fig = px.histogram(
    data,
    x='Class',
    title='Histogram of Class Imbalance',
    labels={'Class': 'Class (0=Benign, 1=Malicious)'},
    color='Class',
    color_discrete_map={0: 'royalblue', 1: 'crimson'},
)
fig.update_layout(
    width=500,
    height=350,
    bargap=0.3,
    showlegend=False
)
fig.show()

In [85]:
data.head()

,Payloads,Class
0,http://www.nwce.gov.uk/search_process.php?keyword=%22%3e%3cscript%3ealert%28document.cookie%29%3b%3c<br>%2fscript%3e,Malicious
1,http://www.manchester.gov.uk/site/scripts/google_results.php?q=%22%3e%253cscript%3ealert%28document.<br>cookie%29%3b%253c%2fscript%3e&amp;btng=search&amp;ie=&amp;site=&amp;output=xml&amp;client=&amp;lr=&amp;oe=&amp;filter=0,Malicious
2,http://www.ldsmissions.com/us/index.php?action=missionary.info%3cmarquee%3epappy%3c/marquee%3e&amp;missi<br>onary_id=69,Malicious
3,http://education.powys.gov.uk/english/adult_ed/register.php?lforenam=\%22%3e%3cscript%3ealert(docume<br>nt.cookie);%3c/script%3e&amp;subdwell=&amp;dwelling=&amp;streetnm=&amp;locality=&amp;hometown=&amp;postcode=&amp;datebrth=&amp;learn<br>gen=&amp;ethnicor=&amp;tel_numb=&amp;tel_mob=&amp;email_add=&amp;email_add2=&amp;agree_info=&amp;username=&amp;password=&amp;err=sur,Malicious
4,http://www.northwarks.gov.uk/site/scripts/google_results.php?q=%22%3e%253cscript%3ealert%28document.<br>cookie%29%3b%253c%2fscript%3e&amp;btng=search&amp;ie=&amp;site=&amp;output=xml&amp;client=&amp;lr=&amp;oe=&amp;filter=0,Malicious


In [86]:
data.tail()

,Payloads,Class
43212,modo=registro&login=willy&password=c53p7t25064&nombre=%3cscript%3ealert%28%22paros%22%29%3b%3c%2fscript%3e&apellidos=pujos+janeiro&email=eberle%40baybay.tg&dni=85025185g&direccion=de+la+vega+4%2c+&ciudad=casasimarro&cp=21340&provincia=a+coru%f1a&ntc=7642131917731646&b1=registrar,Malicious
43213,modo=registro&login=lorrayne&password=anofele6&nombre=neliana&apellidos=monllor+mascort&email=tsaganeas%40mccomptables.my&dni=26431261y&direccion=calle+begonia+47%2c+&ciudad=recuenco%2c+el&cp=44168&provincia=salamanca&ntc=7168408229054610paros%2522%2bstyle%253d%2522background%253aurl%2528javascript%253aalert%2528%2527paros%2527%2529%2529%26id%3d2&b1=registrar,Malicious
43214,errormsg=paros%2522%2520style%3d%2522background%3aurl%28javascript%3aalert%28%27paros%27%29%29,Malicious
43215,modo=registro&login=corissa&password=cipresino&nombre=cenobio&apellidos=la+ringa+chinchilla&email=plate%40pecher.mx&dni=16052504e&direccion=torres+quevedo%2c+143+10%3fb&ciudad=casas+de+juan+n%fa%f1ez&cp=31480&provincia=%253cscript%253ealert%2528%2522paros%2522%2529%253b%253c%252fscript%253e&ntc=1974630082144363&b1=registrar,Malicious
43216,modo=registro&login=carolyne&password=1%f35i7am41t0&nombre=ram%f3n&apellidos=chambueta+deben&email=%3cscript%3ealert%28%22paros%22%29%3b%3c%2fscript%3e&dni=10042805q&direccion=calle+falconera%2c+173+&ciudad=trefacio&cp=31227&provincia=salamanca&ntc=1976494121699035&b1=registrar,Malicious


In [87]:
# drop duplicate rows
data = data.drop_duplicates()

In [88]:
data.count()

Payloads    42671
Class       42671
dtype: int64

In [89]:
# Change class labels to integers
data['Class'] = data['Class'].map({'Malicious': 1, 'Benign': 0})

In [90]:
data.describe()

,Class
count,42671.000000
mean,0.342223
std,0.474459
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


#### Analysis
The mean is less than 0.5, therefore the dataset is slightly imbalanced (dataset consists of 75% of benign data) thus our model would lean towards predicting benign data than malicious data.

Thus, we can either augment the dataset, use resampling techniques, use class weighting and etc.

In [91]:
data.describe(include='object')

,Payloads
count,42671
unique,42671
top,http://www.nwce.gov.uk/search_process.php?keyword=%22%3e%3cscript%3ealert%28document.cookie%29%3b%3c<br>%2fscript%3e
freq,1


In [92]:
# Count the number of 1s and 0s in the 'Class' column
class_counts = data['Class'].value_counts()
print("Class counts:\n", class_counts)

Class counts:
 Class
0    28068
1    14603
Name: count, dtype: int64


In [93]:
data.head()

,Payloads,Class
0,http://www.nwce.gov.uk/search_process.php?keyword=%22%3e%3cscript%3ealert%28document.cookie%29%3b%3c<br>%2fscript%3e,1
1,http://www.manchester.gov.uk/site/scripts/google_results.php?q=%22%3e%253cscript%3ealert%28document.<br>cookie%29%3b%253c%2fscript%3e&amp;btng=search&amp;ie=&amp;site=&amp;output=xml&amp;client=&amp;lr=&amp;oe=&amp;filter=0,1
2,http://www.ldsmissions.com/us/index.php?action=missionary.info%3cmarquee%3epappy%3c/marquee%3e&amp;missi<br>onary_id=69,1
3,http://education.powys.gov.uk/english/adult_ed/register.php?lforenam=\%22%3e%3cscript%3ealert(docume<br>nt.cookie);%3c/script%3e&amp;subdwell=&amp;dwelling=&amp;streetnm=&amp;locality=&amp;hometown=&amp;postcode=&amp;datebrth=&amp;learn<br>gen=&amp;ethnicor=&amp;tel_numb=&amp;tel_mob=&amp;email_add=&amp;email_add2=&amp;agree_info=&amp;username=&amp;password=&amp;err=sur,1
4,http://www.northwarks.gov.uk/site/scripts/google_results.php?q=%22%3e%253cscript%3ealert%28document.<br>cookie%29%3b%253c%2fscript%3e&amp;btng=search&amp;ie=&amp;site=&amp;output=xml&amp;client=&amp;lr=&amp;oe=&amp;filter=0,1


In [94]:
data.isnull().sum()

Payloads    0
Class       0
dtype: int64

In [95]:
# Check structure of benign payloads
data['Payloads'][data['Class'] == 0].head(10)

10000    function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start': new date().gettime(),event:'gtm.js'});var f=d.getelementsbytagname(s)[0], j=d.createelement(s),dl=l!='datalayer'?'&l='+l:'';j.async=true;j.src='../../../www.googletagmanager.com/gtm5445.html?id='+i+dl;f.parentnode.insertbefore(j,f);})(window,document,'script','datalayer','gtm-5p2thv');
10001                                                                                                                                                                                                                                                                                                   http://www.cartoonnetwork.co.uk/show/ok-ko-lets-be-heroes
10002                                                                                                                                                                                                                                                type="text/javascript" src="../../../pcache.ale

In [96]:
def is_probably_url_or_query(s):
    """Check if the string resembles a URL or query-like input."""
    if not isinstance(s, str) or not s.strip():
        return False
    s = s.strip()
    # Match: full URLs, query strings, or key=value pairs
    return bool(re.search(r'(https?://|www\.|[\w]+=)', s, re.IGNORECASE))

def deep_decode(s, rounds=3):
    """Decode URL-encoded and HTML-encoded text multiple times."""
    for _ in range(rounds):
        s = urllib.parse.unquote_plus(s)
        s = html.unescape(s)
    return s

def extract_malicious_payload_safe(entry):
    try:
        if pd.isnull(entry) or not is_probably_url_or_query(entry):
            return entry.strip() if isinstance(entry, str) else entry

        decoded = deep_decode(entry)

        # Try parsing as full URL
        parsed = urlparse(decoded)
        if parsed.query:
            query_items = parse_qsl(parsed.query)
        else:
            # Fallback: parse key=value pairs in entire string
            query_items = parse_qsl(decoded)

        payload_parts = [v for k, v in query_items if v and not v.isnumeric()]

        # Optional: filter only suspicious-looking values (e.g., contain script, javascript)
        suspicious = [p for p in payload_parts if re.search(r'(script|alert|onerror|javascript:|<|>)', p, re.IGNORECASE)]

        return " ".join(suspicious if suspicious else payload_parts).strip() or decoded

    except Exception:
        return entry.strip() if isinstance(entry, str) else entry

# Apply to Payloads column
data['Payloads'] = data['Payloads'].apply(extract_malicious_payload_safe)
data.head(10)

,Payloads,Class
0,"""><script>alert(document.cookie);<<br>/script>",1
1,"""><script>alert(document.<br>cookie);</script>",1
2,missionary.info<marquee>pappy</marquee>,1
3,"\""><script>alert(docume<br>nt.cookie);</script>",1
4,"""><script>alert(document.<br>cookie);</script>",1
5,'><iframe src=http://google.com>,1
6,"""><script>alert(document.co<br>okie);</script>",1
7,'><iframe src=http://google.com>,1
8,"""><script>alert(document.cookie);</script>",1
9,"""><iframe src=http://google.com><font color=red%<br>3e",1


In [97]:
data.isnull().sum()

Payloads    0
Class       0
dtype: int64

In [98]:
# Count the number of 1s and 0s in the 'Class' column
class_counts = data['Class'].value_counts()
print("Class counts:\n", class_counts)

Class counts:
 Class
0    28068
1    14603
Name: count, dtype: int64


In [99]:
# drop duplicate rows again after processing
data = data.drop_duplicates()

In [100]:
# Count the number of 1s and 0s in the 'Class' column
class_counts = data['Class'].value_counts()
print("Class counts:\n", class_counts)

Class counts:
 Class
0    16672
1     8588
Name: count, dtype: int64


In [101]:
data.describe()

,Class
count,25260.000000
mean,0.339984
std,0.473713
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


## Extracting dataset to simulate via DVWA

In [102]:
# Save Malicious Payloads to a new CSV file
malicious_payloads = data[data['Class'] == 1]['Payloads']
# save csv locally C:\Users\tmros\OneDrive\Desktop\Modifying Datasets\XSS data\dataset analysis
malicious_payloads.to_csv("C:\\Users\\tmros\\OneDrive\\Desktop\\Modifying Datasets\\XSS data\\dataset analysis\\malicious_payloads.csv", index=False, encoding='utf-8')
# Save benign Payloads to a new CSV file
benign_payloads = data[data['Class'] == 0]['Payloads']
benign_payloads.to_csv("C:\\Users\\tmros\\OneDrive\\Desktop\\Modifying Datasets\\XSS data\\dataset analysis\\benign_payloads.csv", index=False, encoding='utf-8')

Following the automated injection of the dataset into DVWA using XSStrike, it was confirmed that the malicious payloads effectively exploited the reflected XSS vulnerability within the application. In contrast, analysis of the benign payloads revealed that 553 were flagged as potentially vulnerable by XSStrike. However, subsequent manual validation through DVWA demonstrated that these payloads were not inherently malicious. Rather, XSStrike identified them based on syntactic patterns or components commonly associated with malicious XSS attacks. While these fragments may contribute to a complete exploit under certain conditions, they do not, in isolation, constitute a security threat.

## Chosen Approach: Data Augmentation with XSStrike
To address the class imbalance in the dataset, additional malicious samples will be generated using a controlled testing environment. The DVWA (Damn Vulnerable Web Application), hosted on VirtualBox, is used as the target for simulated XSS attacks.

An open-source tool, XSStrike, will be employed to launch these simulated attacks. A custom script will be developed to intercept, extract, and log the injected XSS payloads from the attack traffic. These verified malicious payloads will then be labeled and added to the original dataset, thereby enriching the minority class (XSS samples) and improving model robustness.


We artificially collected additional data to augment the malicious payloads data, the following options area considered:

Option 1: XSStrike generated payloads used to test vulnerabilities

Option 2: Payloads from github repo, https://github.com/danielmiessler/SecLists/blob/master/Fuzzing/XSS/robot-friendly/XSS-Fuzzing.txt


### Cleaning additional datasets

In [103]:
# Load Option xsstrike payloads
file_path2 = "C:\\Users\\tmros\\OneDrive\\Desktop\\Modifying Datasets\\XSS data\\xsstrike_payloads.csv"
op1 = pd.read_csv(file_path2)
op1.head()

,target,payload
0,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,<deTAils%0aONtoGGle%0a=%0aconfirm()%0dx> [0m
1,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,"<deTAiLs%0aOnpOiNtErenteR%09=%09a=prompt,a()// [0m"
2,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,<d3V/+/onMOuSeoVeR%0a=%0a[8].find(confirm)%0dx>v3dm0s [0m
3,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,<DEtAILs%09ONPoIntERenTER%09=%09confirm()> [0m
4,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,<hTml%0aONPOinTeReNTeR%09=%09(prompt)``> [0m


In [104]:
op1 = op1.drop('target', axis=1)
op1.head()

,payload
0,<deTAils%0aONtoGGle%0a=%0aconfirm()%0dx> [0m
1,"<deTAiLs%0aOnpOiNtErenteR%09=%09a=prompt,a()// [0m"
2,<d3V/+/onMOuSeoVeR%0a=%0a[8].find(confirm)%0dx>v3dm0s [0m
3,<DEtAILs%09ONPoIntERenTER%09=%09confirm()> [0m
4,<hTml%0aONPOinTeReNTeR%09=%09(prompt)``> [0m


In [105]:
# add a new column to the dataset called Class
op1['Class'] = 1
# change payload to Payloads
op1.rename(columns={'payload': 'Payloads'}, inplace=True)

# remove ' [0m' from the Payloads column
op1['Payloads'] = op1['Payloads'].str.replace(' [0m', '', regex=False)
op1.head()

,Payloads,Class
0,<deTAils%0aONtoGGle%0a=%0aconfirm()%0dx>,1
1,"<deTAiLs%0aOnpOiNtErenteR%09=%09a=prompt,a()//",1
2,<d3V/+/onMOuSeoVeR%0a=%0a[8].find(confirm)%0dx>v3dm0s,1
3,<DEtAILs%09ONPoIntERenTER%09=%09confirm()>,1
4,<hTml%0aONPOinTeReNTeR%09=%09(prompt)``>,1


In [106]:
# normalize the Payloads column
op1['Payloads'] = op1['Payloads'].str.lower()


In [107]:
def url_decode(payload):
    return urllib.parse.unquote(payload)

def hex_unescape(payload):
    # Replace \xNN sequences with the actual character
    return re.sub(r'\\x([0-9A-Fa-f]{2})', lambda m: chr(int(m.group(1), 16)), payload)

def normalize_whitespace(payload):
    return " ".join(payload.split())

def normalize_payload(payload):
    step1 = url_decode(payload)
    step2 = hex_unescape(step1)
    step3 = normalize_whitespace(step2)
    return step3

# Apply normalization to the Payloads column
op1['Payloads'] = op1['Payloads'].apply(normalize_payload)
op1.head()

,Payloads,Class
0,<details ontoggle = confirm() x>,1
1,"<details onpointerenter = a=prompt,a()//",1
2,<d3v/+/onmouseover = [8].find(confirm) x>v3dm0s,1
3,<details onpointerenter = confirm()>,1
4,<html onpointerenter = (prompt)``>,1


In [108]:
# Remove 'v3dm0s' from the Payloads column
op1['Payloads'] = op1['Payloads'].str.replace('v3dm0s', '', regex=False)
op1.head()

,Payloads,Class
0,<details ontoggle = confirm() x>,1
1,"<details onpointerenter = a=prompt,a()//",1
2,<d3v/+/onmouseover = [8].find(confirm) x>,1
3,<details onpointerenter = confirm()>,1
4,<html onpointerenter = (prompt)``>,1


In [109]:
op1.isnull().sum()

Payloads    0
Class       0
dtype: int64

In [110]:
op1.describe()

,Class
count,3072.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [111]:

op1 = op1.drop_duplicates(subset=['Payloads'],keep='first')
op1.count()

Payloads    480
Class       480
dtype: int64

There's 2592 duplicated payloads in option 1, thus we have 480 unique payloads

In [112]:
file_path3 = "C:\\Users\\tmros\\OneDrive\\Desktop\\Modifying Datasets\\XSS data\\xsstrike_payloads_seclists.csv"
op2 = pd.read_csv(file_path3)
op2.head()

,target,payload
0,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,</script> [0m
1,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,"<script\x3Etype=""text/javascript"">javascript:alert(1);</script> [0m"
2,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,"<script\x0Dtype=""text/javascript"">javascript:alert(1);</script> [0m"
3,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,"<script\x09type=""text/javascript"">javascript:alert(1);</script> [0m"
4,http://127.0.0.1/DVWA/vulnerabilities/xss_r/?name=test,"<script\x0Ctype=""text/javascript"">javascript:alert(1);</script> [0m"


In [113]:
# drop 'target' column
op2 = op2.drop('target', axis=1)

# add a new column to the dataset called Class
op2['Class'] = 1
# change payload to Payloads
op2.rename(columns={'payload': 'Payloads'}, inplace=True)

# remove ' [0m' from the Payloads column
op2['Payloads'] = op2['Payloads'].str.replace(' [0m', '', regex=False)
op2.head()

,Payloads,Class
0,</script>,1
1,"<script\x3Etype=""text/javascript"">javascript:alert(1);</script>",1
2,"<script\x0Dtype=""text/javascript"">javascript:alert(1);</script>",1
3,"<script\x09type=""text/javascript"">javascript:alert(1);</script>",1
4,"<script\x0Ctype=""text/javascript"">javascript:alert(1);</script>",1


In [114]:
op2.isnull().sum()

Payloads    0
Class       0
dtype: int64

In [115]:
op2.describe()

,Class
count,3762.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [116]:
# Remove duplicates from the Payloads column
# This will keep only the first occurrence of each unique payload
op2 = op2.drop_duplicates(subset=['Payloads'])
op2.count()

Payloads    538
Class       538
dtype: int64

There's 3224 duplicated payloads, thus 538 payloads are unique.

In [117]:
# combine the two datasets
combined_data = pd.concat([op1, op2], ignore_index=True)

In [118]:
combined_data.describe()

,Class
count,1018.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [119]:
combined_data = combined_data.drop_duplicates(subset=['Payloads'], keep='first')
combined_data.count()

Payloads    1018
Class       1018
dtype: int64

In [120]:

# Final updated dataframe
# Combine the original data with the new datasets
tdf = pd.concat([data, combined_data], ignore_index=True)
tdf

,Payloads,Class
0,"""><script>alert(document.cookie);<<br>/script>",1
1,"""><script>alert(document.<br>cookie);</script>",1
2,missionary.info<marquee>pappy</marquee>,1
3,"\""><script>alert(docume<br>nt.cookie);</script>",1
4,'><iframe src=http://google.com>,1
...,...,...
26273,<IMG SRC=javascript:alert(&quot;XSS&quot;)>,1
26274,<IMG SRC=JaVaScRiPt:alert('XSS')>,1
26275,"<IMG SRC=""javascript:alert('XSS');"">",1
26276,"</title></style></textarea></script></xmp><svg/onload='+/""/+/onmouseover=1/+/[*/[]/+alert(1)//'>",1


In [121]:
# Histogram of Class imbalance with custom colors and compact layout
fig = px.histogram(
    tdf,
    x='Class',
    title='Histogram of Class Imbalance',
    labels={'Class': 'Class (0=Benign, 1=Malicious)'},
    color='Class',
    color_discrete_map={0: 'royalblue', 1: 'crimson'},
)
fig.update_layout(
    width=500,
    height=350,
    bargap=0.3,
    showlegend=False
)
fig.show()

# Choosing key Performance Metric

In [122]:
tdf.isnull().sum()

Payloads    0
Class       0
dtype: int64

There's no null entries

In [123]:
# drop rows with missing values
tdf = tdf.dropna(subset=['Payloads'])

In [124]:
# Copying dataset to mod_data, where
# we will make the changes, keeping
# original dataframe 'data' as it is.
mod_tdf = tdf.copy()

In [125]:
tdf['Class'].value_counts()

Class
0    16672
1     9606
Name: count, dtype: int64

In [126]:
# converting 'Class' to boolean column 'XSS'
# XSS = True -> Query is XSS
# XSS = False -> Query is not XSS
mod_tdf['XSS'] = mod_tdf['Class'].apply(bool)
mod_tdf.drop('Class', axis=1, inplace=True)

In [127]:
mod_tdf['XSS'].value_counts()

XSS
False    16672
True      9606
Name: count, dtype: int64

### Removing duplicates

In [128]:
mod_tdf.duplicated().sum()

np.int64(7)

In [129]:
mod_tdf[mod_tdf.duplicated()==True]['XSS'].value_counts()

XSS
True    7
Name: count, dtype: int64

In [130]:
mod_tdf.drop_duplicates(inplace=True)
mod_tdf.shape

(26271, 2)

In [131]:
mod_tdf['XSS'].value_counts()

XSS
False    16672
True      9599
Name: count, dtype: int64

In [132]:
mod_tdf.to_csv('xss_dataset.csv', index=False)